In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.statistical import CopulaSimulation
from teslakit.mda import MaxDiss_Simplified_NoThreshold


In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('GUAM')


In [3]:
# --------------------------------------
# load data and set parameters

_, xds_TCs_r2_params = db.Load_TCs_r2()  # TCs parameters inside radius 2

# TCs random generation and MDA parameters
num_sim_rnd = 100000
num_sel_mda = 1000


In [4]:
# --------------------------------------
# Probabilistic simulation Historical TCs

# aux function
def FixPareto(var):
    'Fix data. It needs to start at 0 for Pareto adjustment '
    var = var.astype(float)
    mx = np.amax(var)
    aux = mx + np.absolute(var - mx)
    var_pareto = aux - np.amin(aux) + 0.00001
    
    return var_pareto, np.amin(aux)


# use small radius parameters (4º)
pmean = xds_TCs_r2_params.pressure_mean.values[:]
pmin = xds_TCs_r2_params.pressure_min.values[:]
gamma = xds_TCs_r2_params.gamma.values[:]
delta = xds_TCs_r2_params.delta.values[:]
vmean = xds_TCs_r2_params.velocity_mean.values[:]

# fix pressure for pareto adjustment (start at 0)
pmean_p, auxmin_pmean = FixPareto(pmean)
pmin_p, auxmin_pmin = FixPareto(pmin)

# join storm parameters for copula simulation
storm_params = np.column_stack(
    (pmean_p, pmin_p, gamma, delta, vmean)
)

# statistical simulate PCs using copulas 
kernels = ['GPareto', 'GPareto', 'ECDF', 'ECDF', 'ECDF']
storm_params_sim = CopulaSimulation(storm_params, kernels, num_sim_rnd)

# store simulated storms - parameters 
xds_TCs_r2_sim_params = xr.Dataset(
    {
        'pressure_mean':(('storm'), storm_params_sim[:,0] + auxmin_pmean),
        'pressure_min':(('storm'), storm_params_sim[:,1] + auxmin_pmin),
        'gamma':(('storm'), storm_params_sim[:,2]),
        'delta':(('storm'), storm_params_sim[:,3]),
        'velocity_mean':(('storm'), storm_params_sim[:,4]),
    },
    coords = {
        'storm':(('storm'), np.arange(num_sim_rnd))    
    },
)
print(xds_TCs_r2_sim_params)

db.Save_TCs_r2_sim_params(xds_TCs_r2_sim_params)


<xarray.Dataset>
Dimensions:        (storm: 100000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables:
    pressure_mean  (storm) float64 1.031e+03 1.012e+03 ... 1.015e+03 1.079e+03
    pressure_min   (storm) float64 1.033e+03 1.013e+03 ... 1.015e+03 1.171e+03
    gamma          (storm) float64 92.69 104.9 105.3 125.8 ... 103.0 0.0 92.63
    delta          (storm) float64 113.1 107.5 102.1 100.6 ... 89.52 56.81 208.2
    velocity_mean  (storm) float64 34.6 16.19 25.45 20.4 ... 32.11 27.76 16.12


In [5]:
# --------------------------------------
# MaxDiss classification

# get simulated parameters  
pmean_s = xds_TCs_r2_sim_params.pressure_mean.values[:]
pmin_s = xds_TCs_r2_sim_params.pressure_min.values[:]
gamma_s = xds_TCs_r2_sim_params.gamma.values[:]
delta_s = xds_TCs_r2_sim_params.delta.values[:]
vmean_s = xds_TCs_r2_sim_params.velocity_mean.values[:]

# subset, scalar and directional indexes
data_mda = np.column_stack((pmean_s, pmin_s, vmean_s, delta_s, gamma_s))
ix_scalar = [0,1,2]
ix_directional = [3,4]

centroids = MaxDiss_Simplified_NoThreshold(
    data_mda, num_sel_mda, ix_scalar, ix_directional 
)


# store MDA storms - parameters 
xds_TCs_r2_MDA_params = xr.Dataset(
    {
        'pressure_mean':(('storm'), centroids[:,0]),
        'pressure_min':(('storm'), centroids[:,1]),
        'gamma':(('storm'), centroids[:,2]),
        'delta':(('storm'), centroids[:,3]),
        'velocity_mean':(('storm'), centroids[:,4]),
    },
    coords = {
        'storm':(('storm'), np.arange(num_sel_mda))    
    },
)
print(xds_TCs_r2_MDA_params)

db.Save_TCs_r2_mda_params(xds_TCs_r2_MDA_params)



MaxDiss waves parameters: 100000 --> 1000

   MDA centroids: 1000/1000

<xarray.Dataset>
Dimensions:        (storm: 1000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    pressure_mean  (storm) float64 1.745e+03 1.013e+03 ... 1.107e+03 1.121e+03
    pressure_min   (storm) float64 1.43e+03 1.014e+03 ... 1.078e+03 1.141e+03
    gamma          (storm) float64 32.24 10.75 46.35 4.798 ... 10.9 20.42 23.58
    delta          (storm) float64 97.18 280.1 167.8 51.18 ... 230.5 140.9 106.3
    velocity_mean  (storm) float64 109.7 267.9 90.03 101.0 ... 123.6 231.5 33.94
